# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
df=pd.read_csv("IMDB Dataset.csv",)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def rem_stopwords(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    text=" ".join(filtered_sentence)
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [lemmatizer.lemmatize(w) for w in word_tokens]
    text=" ".join(filtered_sentence)
    return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def clean_text(text):
    text = text.lower()
    clean = re.compile('<.*?>')
    text = re.sub(clean, ' ', text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'\W+', ' ', text)
    text = rem_stopwords(text)
    text = lemmatize(text)
    return text
df["review"] = df["review"].map(lambda com : clean_text(com))
df

,review,sentiment
0,one reviewer mentioned watching 1 oz episode h...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake think zombie ...,negative
4,petter mattei love time money visually stunnin...,positive
...,...,...
49995,thought movie right good job creative original...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,catholic taught parochial elementary school nu...,negative
49998,going disagree previous comment side maltin on...,negative


In [6]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
sentence=[]
s0=df['review'].tolist()
for i in range(len(s0)):
    word_tokens = word_tokenize(s0[i])
    sentence.append(word_tokens)
# Data distribution w.r.t labels
print(df.groupby('sentiment').count())

# Average Length of sentence
review_len = [len(s) for s in sentence]
print("\nAverage length of review : {}".format(np.mean(review_len)))
print("Max length of review : {}".format(np.max(review_len)))
print("Min length of review : {}".format(np.min(review_len)))
print("Standard dev of length of review : {}".format(np.std(review_len)))

           review
sentiment        
negative    25000
positive    25000

Average length of review : 119.81342
Max length of review : 1437
Min length of review : 3
Standard dev of length of review : 90.42191597120468


# Naive Bayes classifier

In [7]:
# get reviews column from df
reviews = df['review']

# get labels column from df
labels = df['sentiment']

In [8]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print(encoder.classes_)

['negative' 'positive']


In [9]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
X=df.review

train_sentences, test_sentences, train_labels, test_labels=train_test_split(X,encoded_labels,test_size=0.2,random_state=0,
                                                                            stratify=encoded_labels)

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
#max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
def dummy(doc):
    return doc
sentences=[]
s=train_sentences.tolist()
for i in range(len(s)):
    word_tokens = word_tokenize(s[i])
    sentences.append(word_tokens)
vec = CountVectorizer(tokenizer=dummy,preprocessor=dummy,max_features = 3000)
X = vec.fit_transform(sentences)
V=X.toarray()
Vocab=vec.vocabulary_
Vocab

{'portrayed': 2009,
 'film': 1023,
 'seems': 2328,
 'today': 2715,
 'exactly': 914,
 'main': 1623,
 'character': 454,
 'make': 1628,
 'entire': 879,
 'story': 2535,
 'tragic': 2745,
 'allows': 111,
 'stand': 2499,
 'like': 1557,
 'tony': 2724,
 'said': 2271,
 'love': 1601,
 'happens': 1228,
 'need': 1796,
 'explanation': 943,
 'simply': 2401,
 'despite': 725,
 'wanting': 2886,
 'level': 1549,
 'happened': 1226,
 'social': 2444,
 'timing': 2711,
 'relationship': 2173,
 'coming': 536,
 'far': 984,
 'enough': 873,
 'sitting': 2417,
 'young': 2994,
 'couple': 608,
 'still': 2525,
 'conversation': 589,
 'throughout': 2697,
 'pain': 1907,
 'lost': 1595,
 'missed': 1723,
 'opportunity': 1879,
 'gross': 1193,
 'deeper': 695,
 'emotion': 855,
 'made': 1617,
 'realize': 2146,
 'much': 1760,
 'well': 2918,
 'work': 2967,
 'wasted': 2897,
 'local': 1577,
 'audience': 207,
 'good': 1165,
 'men': 1684,
 'great': 1183,
 'action': 61,
 'movie': 1756,
 'lot': 1596,
 'twist': 2783,
 'turn': 2776,
 'jame

In [11]:
Vocab["film"]

1023

In [12]:
# Use laplace smoothing for words in test set not present in vocab of train set
pos=0
neg=0
L=len(V)
for i in range(L):
    if(train_labels[i]==1):
        pos=pos+1
    else:
        neg=neg+1
vocab_size=3000
pos_words=0
neg_words=0
for i in range(L):
    if(train_labels[i]==1):
        for j in range(vocab_size):
            pos_words=pos_words+V[i][j]
    else:
        for j in range(vocab_size):
            neg_words=neg_words+V[i][j]
p=pos/(pos+neg)
n=neg/(pos+neg)
c_neg=np.zeros(3000)
c_pos=np.zeros(3000)
for i in range(L):
    if(train_labels[i]==1):
        c_pos=c_pos+V[i]
    else:
        c_neg=c_neg+V[i]
def prob_pos(word):
    count=0
    value=Vocab.get(word)
    if(value!=None):
        count=count+c_pos[Vocab[word]]
    prob=(count+1)/(pos_words+vocab_size)
    return prob
def prob_neg(word):
    count=0
    value=Vocab.get(word)
    if(value!=None):
        count=count+c_neg[Vocab[word]]
    prob=(count+1)/(neg_words+vocab_size)
    return prob

In [13]:
# Build the model. Don't use the model from sklearn
from tqdm.notebook import tqdm
def Naive_Bayes(sentences):
    pred=[]
    s=sentences.tolist()
    for i in tqdm(range(len(s))):
        word_tokens = word_tokenize(s[i])
        prob_p=1
        prob_n=1
        for w in word_tokens:
            prob_p=prob_p*prob_pos(w)
            prob_n=prob_n*prob_neg(w)
            if(prob_p<1e-100 or prob_n<1e-100):
                prob_p*=1e100
                prob_n*=1e100
        prob_p=prob_p*p
        prob_n=prob_n*n
        if(prob_p>prob_n):
            pred.append(1)
        else:
            pred.append(0)
    return pred

In [14]:
# Test the model on test set and report Accuracy

Pred=Naive_Bayes(test_sentences)

In [15]:
print(classification_report(test_labels,Pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85      5000
           1       0.84      0.85      0.85      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



# *LSTM* based Classifier

Use the above train and test splits.

In [16]:
# Hyperparameters of the model
vocab_size = 3000# choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length =1440 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [17]:
# tokenize sentences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [18]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1440, 100)         300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [19]:
num_epochs = 5
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 1767s 2s/step - loss: 0.4033 - accuracy: 0.8183 - val_loss: 0.3144 - val_accuracy: 0.8730
Epoch 2/5
1125/1125 [==============================] - 1785s 2s/step - loss: 0.2841 - accuracy: 0.8861 - val_loss: 0.3007 - val_accuracy: 0.8832
Epoch 3/5
1125/1125 [==============================] - 1783s 2s/step - loss: 0.2485 - accuracy: 0.9032 - val_loss: 0.3248 - val_accuracy: 0.8773
Epoch 4/5
1125/1125 [==============================] - 1782s 2s/step - loss: 0.2247 - accuracy: 0.9144 - val_loss: 0.3360 - val_accuracy: 0.8648
Epoch 5/5
1125/1125 [==============================] - 1791s 2s/step - loss: 0.2145 - accuracy: 0.9189 - val_loss: 0.3319 - val_accuracy: 0.8730


In [20]:
# Calculate accuracy on Test data

# Get probabilities
prediction = model.predict(test_padded)

# Get labels based on probability 1 if p>= 0.5 else 0
test_preds_lstm = np.int32(prediction >= 0.5)

# Accuracy : one can use classification_report from sklearn
print("Performance of LSTM Classifier - \n")
print(classification_report(test_labels, test_preds_lstm))

Performance of LSTM Classifier - 

              precision    recall  f1-score   support

           0       0.88      0.87      0.87      5000
           1       0.87      0.88      0.88      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## Get predictions for random examples

In [21]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# LSTM Classifier - 
# convert to a sequence
sequences = tokenizer.texts_to_sequences([clean_text(sent) for sent in sentence])

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
probas = model.predict(padded)
print("LSTM Classifier predicted probabilities - ")
print([p[0] for p in probas])

# Get labels based on probability 1 if p>= 0.5 else 0
preds = np.int32(probas >= 0.5)

label_map = {1: "positive", 0: "negative"}
print("\nLSTM classifier predictions - ")
labels = [label_map[p[0]] for p in preds]
print(labels)


LSTM Classifier predicted probabilities - 
[0.92379427, 0.10890445, 0.09004697]

LSTM classifier predictions - 
['positive', 'negative', 'negative']


In [22]:
# Naive Bayes Classifier - 
# convert to a sequence
sequences = [clean_text(sent) for sent in sentence]
Seq=pd.DataFrame(sequences,columns=['sent'])
Seq=Seq.sent
# Get labels 
preds = Naive_Bayes(Seq)

label_map = {1: "positive", 0: "negative"}
print("\nNaive-Bayes classifier predictions - ")
labels = [label_map[p] for p in preds]
print(labels)



Naive-Bayes classifier predictions - 
['positive', 'negative', 'negative']
